<a href="https://colab.research.google.com/github/baeck3505/BA_dataFrame/blob/main/ch02_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%8B%A4%EB%93%AC%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 학습 목표
###1.결측값 찾기 및 처리
###2.중복값 찾기 및 처리
###3.이상점 찾기 및 처리


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
airbnb_df = pd.read_csv('/content/drive/MyDrive/코드잇csv/airbnb.csv')
airbnb_df #에어비엔비 숙소데이트, NaN:결측값

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


In [ ]:
airbnb_df.info() #데이터갯수 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             713 non-null    object 
 1   state          713 non-null    object 
 2   guests         713 non-null    int64  
 3   beds           713 non-null    int64  
 4   property_type  713 non-null    object 
 5   rating         700 non-null    float64
 6   n_reviews      713 non-null    int64  
 7   location       713 non-null    object 
 8   price          713 non-null    int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 50.3+ KB


In [ ]:
airbnb_df.isna() #결측값이면 True, 없으면 False

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
708,False,False,False,False,False,False,False,False,False
709,False,False,False,False,False,False,False,False,False
710,False,False,False,False,False,False,False,False,False
711,False,False,False,False,False,False,False,False,False


In [ ]:
airbnb_df.isna().sum() #컬럼별로 결측값의 합을 계산

,0
id,0
state,0
guests,0
beds,0
property_type,0
rating,13
n_reviews,0
location,0
price,0


In [ ]:
airbnb_df.isna().any(axis=1)
#로우단위로 결측값이 있는지 확인, 즉 한 줄씩 결측값이 있는지를 확인
#결측값이면 True, 없으면 False

,0
0,False
1,True
2,False
3,False
4,False
...,...
708,False
709,False
710,False
711,False


In [ ]:
airbnb_df[airbnb_df.isna().any(axis=1)] #결측값만 있는 데이터를 출력

,id,state,guests,beds,property_type,rating,n_reviews,location,price
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
127,MB7199,Illinois,2,1,Apartment,NaN,0,"Rogers Park, Chicago.",74
189,DE6323,illinois,2,1,Condominium,NaN,0,"South Loop/Printers Row, Chicago.",79
206,ME5527,ILLINOIS,2,1,House,NaN,0,"Garfield Park, Chicago.",52
272,RM9271,illinois,2,1,Apartment,NaN,0,"Lincoln Park, Chicago.",64
275,TA9777,California,2,1,Condominium,NaN,0,"Pacific Heights, San Francisco.",115
366,LP6045,ILLINOIS,2,1,Apartment,NaN,0,"Loop, Chicago.",115
534,DC4208,ILLINOIS,5,3,House,NaN,0,"Little Italy/UIC, Chicago.",100
546,IE7576,illinois,1,1,House,NaN,0,"South Chicago, Chicago.",50
552,DY9962,California,1,1,Condominium,NaN,0,"Valley Village, Los Angeles.",85


#결측값 삭제하기
###1. 결측값 개수가 적어서 삭제해도 큰 영향을 주지 않을 때
###2. 컬럼이나 로우에 있는 값들이 거의 다 결측값일 때


#결측값을 다른 값으로 대체
###1.연속형 숫자 데이터는 평균이나 중간값을 주로 사용
###2. 범주형 데이터는 최빈값을 주로 사용

In [ ]:
airbnb_df.dropna() #결측값 삭제하기

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
5,PP4626,California,2,1,Apartment,92.0,211,"Western Addition/NOPA, San Francisco.",129
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


In [ ]:
#결측값을 평균으로 대체하자
airbnb_df['rating'].mean()

94.82

In [ ]:
rating_mean = airbnb_df['rating'].mean()

In [ ]:
airbnb_df['rating'].fillna(rating_mean)
#결측값에 평균값을 대체, fillna함수사용

,rating
0,96.00
1,94.82
2,100.00
3,75.00
4,88.00
...,...
708,97.00
709,93.00
710,93.00
711,80.00


In [ ]:
airbnb_df['rating'] = airbnb_df['rating'].fillna(rating_mean) #해당데이터들을 기존값에 넣자

NameError: name 'rating_mean' is not defined

In [ ]:
airbnb_df

,rating
0,96.00
1,94.82
2,100.00
3,75.00
4,88.00
...,...
708,97.00
709,93.00
710,93.00
711,80.00


In [ ]:
airbnb_df.isna().sum()

0

#실습 설명
###효준이는 새로운 스마트폰을 구입하기 위해 애플과 삼성의 스마트폰 데이터를 수집했습니다.
###그런데 실수로 특정 모델의 무게를 입력하는 것을 깜빡했다고 하는데요. 다행히 한 가지 모델만 누락되어 있고, 모델명이 같으면 용량이나 색상과 관계없이 무게가 동일하다고 합니다.
###아래 표를 참고해서 cellphone_df에 있는 결측값을 올바른 값으로 채워 주세요. 그 다음에는 cellphone_df에서 2021년에 출시한 스마트폰들의 데이터만 출력해 주세요.

In [ ]:
cellphone_df = pd.read_csv('/content/drive/MyDrive/코드잇csv/cellphone.csv')
cellphone_df

,brand,name,size,price,battery,weight,year
0,apple,iPhone 14 (128GB),"6.1""",1250.0,3729,172.0,2022
1,apple,iPhone 14 (256GB),"6.1""",1400.0,3729,172.0,2022
2,Apple,iPhone 14 (512GB),"6.1""",1700.0,3729,172.0,2022
3,apple,iPhone 14 Plus (128GB),"6.7""",1350.0,4325,203.0,2022
4,APPLE,iPhone 14 Plus (256GB),"6.7""",1500.0,4325,203.0,2022
...,...,...,...,...,...,...,...
65,SAMSUNG,Galaxy S20 Ultra (512GB),"6.9""",1793.0,5000,220.0,2020
66,samsung,Galaxy S20+ (256GB),"6.7""",1353.0,4500,186.0,2020
67,SAMSUNG,Galaxy Z Flip2 (256GB),"6.7""",1650.0,3300,183.0,2020
68,Samsung,Galaxy Z Fold2 (256GB),"7.6""",1892.0,4500,282.0,2020


In [ ]:
cellphone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   brand    70 non-null     object 
 1   name     70 non-null     object 
 2   size     70 non-null     object 
 3   price    70 non-null     float64
 4   battery  70 non-null     int64  
 5   weight   67 non-null     float64
 6   year     70 non-null     int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 4.0+ KB


In [ ]:
cellphone_df.isna().sum()

,0
brand,0
name,0
size,0
price,0
battery,0
weight,3
year,0


In [ ]:
cellphone_df[cellphone_df.isna().any(axis=1)]

,brand,name,size,price,battery,weight,year
37,Apple,iPhone 13 mini (128GB),"5.4""",950.0,2406,NaN,2021
38,Apple,iPhone 13 mini (256GB),"5.4""",1090.0,2406,NaN,2021
39,APPLE,iPhone 13 mini (512GB),"5.4""",1360.0,2406,NaN,2021


In [ ]:
# 결측값 대체
cellphone_df['weight'] = cellphone_df['weight'].fillna(140)
cellphone_df[cellphone_df['year'] == 2021]


,brand,name,size,price,battery,weight,year
26,apple,iPhone 13 (128GB),"6.1""",1090.0,3227,173.0,2021
27,Apple,iPhone 13 (256GB),"6.1""",1230.0,3227,173.0,2021
28,apple,iPhone 13 (512GB),"6.1""",1500.0,3227,173.0,2021
29,APPLE,iPhone 13 Pro (128GB),"6.1""",1350.0,3095,203.0,2021
30,Apple,iPhone 13 Pro (256GB),"6.1""",1490.0,3095,203.0,2021
31,Apple,iPhone 13 Pro (512GB),"6.1""",1760.0,3095,203.0,2021
32,Apple,iPhone 13 Pro (1TB),"6.1""",2030.0,3095,203.0,2021
33,apple,iPhone 13 Pro Max (128GB),"6.7""",1490.0,4352,238.0,2021
34,Apple,iPhone 13 Pro Max (256GB),"6.7""",1630.0,4352,238.0,2021
35,Apple,iPhone 13 Pro Max (512GB),"6.7""",1900.0,4352,238.0,2021


#중복값 찾기

In [ ]:
airbnb_df = pd.read_csv('/content/drive/MyDrive/코드잇csv/airbnb.csv')
airbnb_df

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


In [ ]:
#중복값 확인하기
airbnb_df.duplicated() #True:중복값

,0
0,False
1,False
2,False
3,False
4,False
...,...
708,False
709,False
710,False
711,False


In [ ]:
airbnb_df.duplicated().sum() #중복된 로우가 총 2개

2

In [ ]:
airbnb_df[airbnb_df.duplicated()] #중복된 로우 출력

,id,state,guests,beds,property_type,rating,n_reviews,location,price
493,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
496,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124


In [ ]:
airbnb_df[airbnb_df.duplicated(subset=['id', 'n_reviews', 'price'])]#리스트로도 가능

,id,state,guests,beds,property_type,rating,n_reviews,location,price
493,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
496,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124


In [ ]:
airbnb_df[airbnb_df.duplicated(subset='id')] #id값이 같으면 중복값

,id,state,guests,beds,property_type,rating,n_reviews,location,price
448,LN3479,California,2,2,Apartment,98.0,19,"East Hollywood, Los Angeles.",88
452,LN3479,California,2,2,Apartment,99.0,25,"East Hollywood, Los Angeles.",91
493,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
496,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
527,QO8555,California,7,4,House,80.0,1,"Excelsior, San Francisco.",199
601,BZ3596,Illinois,2,1,House,89.0,30,"Lakeview, Chicago.",119


In [ ]:
airbnb_df[airbnb_df.duplicated(subset='id',keep=False)]
#곂치는 값들은 모두 중복값 전부를 볼수있음

,id,state,guests,beds,property_type,rating,n_reviews,location,price
6,LN3479,California,2,2,Apartment,96.0,15,"East Hollywood, Los Angeles.",88
201,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
273,QO8555,California,7,4,House,85.0,3,"Excelsior, San Francisco.",199
448,LN3479,California,2,2,Apartment,98.0,19,"East Hollywood, Los Angeles.",88
452,LN3479,California,2,2,Apartment,99.0,25,"East Hollywood, Los Angeles.",91
493,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
496,UD8253,MASSACHUSETTS,3,1,Apartment,100.0,1,"Fenway/Kenmore, Boston.",124
527,QO8555,California,7,4,House,80.0,1,"Excelsior, San Francisco.",199
541,BZ3596,Illinois,2,1,Apartment,91.0,28,"Lakeview, Chicago.",119
601,BZ3596,Illinois,2,1,House,89.0,30,"Lakeview, Chicago.",119


#중복값 처리하기
### 중복값 확인하기
airbnb_df.duplicated().sum

###중복값 삭제하기
airbnb_df.drop_duplicates()

In [ ]:
airbnb_df.duplicated().sum()

2

In [ ]:
airbnb_df.drop_duplicates() #중복값 삭제

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


In [ ]:
airbnb_df.drop_duplicates(subset='id')

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


In [ ]:
airbnb_df.drop_duplicates(subset='id', keep='last')
#중복값들 중 마지막 데이터를 제외하고 나머지 중복값은 삭제, first도 가능

# 실습 설명
###효준이가 스마트폰 데이터를 수집할 때 중복으로 여러 번 입력한 데이터가 몇 개 있다고 합니다. cellphone_df에서 중복값은 1개씩만 남기고 나머지는 삭제해 주세요. 참고로 중복으로 입력한 데이터는 모든 컬럼에 있는 값들이 똑같다고 합니다.

###주의 사항: 채점을 위해 가장 마지막 줄에는 cellphone_df라고 입력해서 DataFrame을 출력해 주세요.

In [ ]:
cellphone_dff = pd.read_csv('/content/drive/MyDrive/코드잇csv/cellphone.csv')
cellphone_df

,brand,name,size,price,battery,weight,year
0,apple,iPhone 14 (128GB),"6.1""",1250.0,3729,172.0,2022
1,apple,iPhone 14 (256GB),"6.1""",1400.0,3729,172.0,2022
2,Apple,iPhone 14 (512GB),"6.1""",1700.0,3729,172.0,2022
3,apple,iPhone 14 Plus (128GB),"6.7""",1350.0,4325,203.0,2022
4,APPLE,iPhone 14 Plus (256GB),"6.7""",1500.0,4325,203.0,2022
...,...,...,...,...,...,...,...
65,SAMSUNG,Galaxy S20 Ultra (512GB),"6.9""",1793.0,5000,220.0,2020
66,samsung,Galaxy S20+ (256GB),"6.7""",1353.0,4500,186.0,2020
67,SAMSUNG,Galaxy Z Flip2 (256GB),"6.7""",1650.0,3300,183.0,2020
68,Samsung,Galaxy Z Fold2 (256GB),"7.6""",1892.0,4500,282.0,2020


In [ ]:
cellphone_df.duplicated().sum()

4

In [ ]:
cellphone_df[cellphone_df.duplicated()]

,brand,name,size,price,battery,weight,year
19,samsung,Galaxy S22+ (256GB),"6.6""",1199.0,4500,195.0,2022
23,samsung,Galaxy Z Fold4 (256GB),"7.6""",1998.7,4400,263.0,2022
56,apple,iPhone 12 Pro Max (128GB),"6.7""",1490.0,3687,228.0,2020
57,apple,iPhone 12 Pro Max (128GB),"6.7""",1490.0,3687,228.0,2020


In [ ]:
cellphne_df = cellphone_df.drop_duplicates() #정답
cellphne_df

,brand,name,size,price,battery,weight,year
0,apple,iPhone 14 (128GB),"6.1""",1250.0,3729,172.0,2022
1,apple,iPhone 14 (256GB),"6.1""",1400.0,3729,172.0,2022
2,Apple,iPhone 14 (512GB),"6.1""",1700.0,3729,172.0,2022
3,apple,iPhone 14 Plus (128GB),"6.7""",1350.0,4325,203.0,2022
4,APPLE,iPhone 14 Plus (256GB),"6.7""",1500.0,4325,203.0,2022
...,...,...,...,...,...,...,...
65,SAMSUNG,Galaxy S20 Ultra (512GB),"6.9""",1793.0,5000,220.0,2020
66,samsung,Galaxy S20+ (256GB),"6.7""",1353.0,4500,186.0,2020
67,SAMSUNG,Galaxy Z Flip2 (256GB),"6.7""",1650.0,3300,183.0,2020
68,Samsung,Galaxy Z Fold2 (256GB),"7.6""",1892.0,4500,282.0,2020


# 이상점 찾기

In [ ]:
airbnb_df = pd.read_csv('/content/drive/MyDrive/코드잇csv/airbnb.csv')
airbnb_df

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


In [ ]:
q1 = airbnb_df['price'].quantile(0.25) #1사분위수

In [ ]:
q3 = airbnb_df['price'].quantile(0.75) #3사분위수

In [ ]:
iqr = q3 - q1 #IQR

In [ ]:
lower_limit = q1 - 1.5 * iqr

In [ ]:
upper_limit = q3 + 1.5 * iqr

In [ ]:
airbnb_df['price'] < lower_limit

,price
0,False
1,False
2,False
3,False
4,False
...,...
708,False
709,False
710,False
711,False


In [ ]:
airbnb_df[airbnb_df['price'] < lower_limit] #가격이 너무 낮은 이상점

,id,state,guests,beds,property_type,rating,n_reviews,location,price
13,HB7516,Illinois,1,1,Apartment,94.0,56,"Little Italy/UIC, Chicago.",14
43,IT4655,California,2,8,Hostel,91.0,27,"Westlake, Los Angeles.",14


In [ ]:
airbnb_df[airbnb_df['price'] > upper_limit] #가격이 높은 이상점

,id,state,guests,beds,property_type,rating,n_reviews,location,price
91,FG7538,California,6,3,House,100.0,6,"Marina, San Francisco.",1800
442,ZL3688,California,16,8,Villa,92.0,7,"Malibu, Los Angeles.",1794


In [ ]:
airbnb_df[(airbnb_df['price'] < lower_limit) | (airbnb_df['price'] > upper_limit)]
#최종이상점

,id,state,guests,beds,property_type,rating,n_reviews,location,price
13,HB7516,Illinois,1,1,Apartment,94.0,56,"Little Italy/UIC, Chicago.",14
43,IT4655,California,2,8,Hostel,91.0,27,"Westlake, Los Angeles.",14
91,FG7538,California,6,3,House,100.0,6,"Marina, San Francisco.",1800
442,ZL3688,California,16,8,Villa,92.0,7,"Malibu, Los Angeles.",1794


# 이상점 처리하기

###1. 그대로 두거나
###2. 값을 범위안의 값으로 변경하기
###3. 삭제하기

In [ ]:
condition1 = airbnb_df['price'] >= lower_limit

In [ ]:
condition2 = airbnb_df['price'] <= upper_limit

In [ ]:
airbnb_df[condition1 & condition2]

,id,state,guests,beds,property_type,rating,n_reviews,location,price
0,BF1406,Illinois,2,1,Apartment,96.0,55,"Avondale, Chicago.",48
1,DB7921,MASSACHUSETTS,4,1,Apartment,NaN,0,"South End, Boston.",115
2,OZ5857,ILLINOIS,4,3,Apartment,100.0,51,"Avondale, Chicago.",150
3,HY8797,california,2,1,Apartment,75.0,4,"SoMa, San Francisco.",125
4,VN7858,massachusetts,2,1,House,88.0,23,"West Roxbury, Boston.",64
...,...,...,...,...,...,...,...,...,...
708,XG3289,California,2,1,Apartment,97.0,60,"Venice, Los Angeles.",150
709,ZR5298,california,4,2,Apartment,93.0,3,"Mid-Wilshire, Los Angeles.",100
710,XZ5531,California,2,1,Bungalow,93.0,3,"Redondo Beach, Los Angeles.",100
711,EE8333,ILLINOIS,2,1,House,80.0,15,"Lakeview, Chicago.",115


# 실습 설명
###효준이는 스마트폰 구매 후보에서 가격이 너무 싸거나 비싼 건 제외하려고 합니다. cellphone_df에서 가격을 기준으로 이상점으로 볼 수 있는 데이터는 삭제해 주세요.

In [ ]:
cellphone_df = pd.read_csv('/content/drive/MyDrive/코드잇csv/cellphone.csv')
cellphone_df

,brand,name,size,price,battery,weight,year
0,apple,iPhone 14 (128GB),"6.1""",1250.0,3729,172.0,2022
1,apple,iPhone 14 (256GB),"6.1""",1400.0,3729,172.0,2022
2,Apple,iPhone 14 (512GB),"6.1""",1700.0,3729,172.0,2022
3,apple,iPhone 14 Plus (128GB),"6.7""",1350.0,4325,203.0,2022
4,APPLE,iPhone 14 Plus (256GB),"6.7""",1500.0,4325,203.0,2022
...,...,...,...,...,...,...,...
65,SAMSUNG,Galaxy S20 Ultra (512GB),"6.9""",1793.0,5000,220.0,2020
66,samsung,Galaxy S20+ (256GB),"6.7""",1353.0,4500,186.0,2020
67,SAMSUNG,Galaxy Z Flip2 (256GB),"6.7""",1650.0,3300,183.0,2020
68,Samsung,Galaxy Z Fold2 (256GB),"7.6""",1892.0,4500,282.0,2020


In [ ]:
q1 = cellphone_df['price'].quantile(0.25)

In [ ]:
q3 = cellphone_df['price'].quantile(0.75)

In [ ]:
iqr = q3 - q1

In [ ]:
lower_limit = q1 - 1.5 * iqr

In [ ]:
upper_limit = q3 + 1.5 * iqr

In [ ]:
condition1 = cellphone_df['price'] >= lower_limit

In [ ]:
condition2 = cellphone_df['price'] <= upper_limit

In [ ]:
cellphne_df[condition1 & condition2] #정답

<ipython-input-116-8d30fd49e8ce>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cellphne_df[condition1 & condition2]


,brand,name,size,price,battery,weight,year
0,apple,iPhone 14 (128GB),"6.1""",1250.0,3729,172.0,2022
1,apple,iPhone 14 (256GB),"6.1""",1400.0,3729,172.0,2022
2,Apple,iPhone 14 (512GB),"6.1""",1700.0,3729,172.0,2022
3,apple,iPhone 14 Plus (128GB),"6.7""",1350.0,4325,203.0,2022
4,APPLE,iPhone 14 Plus (256GB),"6.7""",1500.0,4325,203.0,2022
...,...,...,...,...,...,...,...
64,samsung,Galaxy S20 Ultra (256GB),"6.9""",1595.0,5000,220.0,2020
65,SAMSUNG,Galaxy S20 Ultra (512GB),"6.9""",1793.0,5000,220.0,2020
66,samsung,Galaxy S20+ (256GB),"6.7""",1353.0,4500,186.0,2020
67,SAMSUNG,Galaxy Z Flip2 (256GB),"6.7""",1650.0,3300,183.0,2020
